In [1]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [2]:
## intializeing lemmatizer, loading json, creating lists, setting characters to ignore in our training set

lemmatizer = WordNetLemmatizer()

intents = json.loads(open('intents.json').read())

words, classes, documents = [], [] ,[]
ignore_characters = ['?', '!', ',', '.']

## organizing/clean/tokenizing json data

for intent in intents['intents']:
	for pattern in intent['patterns']:
		word_list = nltk.word_tokenize(pattern)
		words.extend(word_list)
		documents.append((word_list, intent['tag']))
		if intent['tag'] not in classes:
			classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_characters]
words = sorted(set(words))

classes = sorted(set(classes))

# print(words)

## Saving data for words/classes into pkl

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

## converting words into numerical values, shuffling, and creating model with numerical data

training = []
output_empty = [0] * len(classes)

for document in documents:
	bag = []
	word_patterns = document[0]
	word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
	for word in words:
		bag.append(1) if word in word_patterns else bag.append(0)

	output_row = list(output_empty)
	output_row[classes.index(document[1])] = 1
	training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

## training neural network

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs= 200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print('Done')

C:\Users\pluss\AppData\Local\Temp/ipykernel_11592/1675308999.py:49: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)
c:\users\pluss\appdata\local\programs\python\python39\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/200
38/38 [==============================] - 0s 1ms/step - loss: 3.5721 - accuracy: 0.1383 
Epoch 2/200
38/38 [==============================] - 0s 1ms/step - loss: 3.1490 - accuracy: 0.2074
Epoch 3/200
38/38 [==============================] - 0s 1ms/step - loss: 2.9257 - accuracy: 0.2287
Epoch 4/200
38/38 [==============================] - 0s 1ms/step - loss: 2.7062 - accuracy: 0.2500
Epoch 5/200
38/38 [==============================] - 0s 1ms/step - loss: 2.6395 - accuracy: 0.2500
Epoch 6/200
38/38 [==============================] - 0s 1ms/step - loss: 2.4174 - accuracy: 0.3670
Epoch 7/200
38/38 [==============================] - 0s 1ms/step - loss: 2.1207 - accuracy: 0.3777
Epoch 8/200
38/38 [==============================] - 0s 2ms/step - loss: 2.0575 - accuracy: 0.4415
Epoch 9/200
38/38 [==============================] - 0s 1ms/step - loss: 1.8965 - accuracy: 0.5000
Epoch 10/200
38/38 [==============================] - 0s 1ms/step - loss: 1.7476 - accuracy: 0.5213
Epoch 11

In [3]:
import os

from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import load_model

from autocorrect import Speller

In [4]:
##loading in data from model

lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())

words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbot_model.h5')

spell = Speller(lang='en')

##cleaning up sentences

def clean_up_sentence(sentence):
	sentence_words = nltk.word_tokenize(sentence)
	sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
	return sentence_words

##getting bag of words

def bag_of_words(sentence):
	sentence_words = clean_up_sentence(sentence)
	bag = [0] * len(words)
	for w in sentence_words:
		for i, word in enumerate(words):
			if word == w:
				bag[i] = 1
	return np.array(bag)

##predicting class

def predict_class(sentence):
	bow = bag_of_words(sentence)
	res = model.predict(np.array([bow]))[0]
	error_threshold = .03
	results = [[i, r] for i, r in enumerate(res) if r > error_threshold]

	results.sort(key=lambda x: x[1], reverse=True)
	return_list = []
	for r in results:
		return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
	return return_list

##getting responce

def get_response(intents_list, intents_json):
	tag = intents_list[0]['intent']
	list_of_intents = intents_json['intents']
	for i in list_of_intents:
		if i['tag'] == tag:
			follow_ups_ = i['followups']
			result = [(random.choice(i['responses0'])+random.choice(i['responses1'])+random.choice(i['responses2'])).format(name), follow_ups_]
			break
	return result

In [5]:
def get_all_responses(var_name):
	list0_, list1_ = [], intents['intents']
	for i in list1_:
		if i['tag'] == var_name:
			for s0_ in i['responses0']:
				for s1_ in i['responses1']:
					for s2_ in i['responses2']:
						list0_.append(s0_ + s1_ + s2_)
	return(list0_)
get_all_responses('training questions')

[]

In [6]:
## variable finder

def var_finder(lst, var_searching, return_var, fail_var):
    var_ = 0
    for i in lst:
        if i == var_searching:
            var_ = 1
            break
    if var_ == 1:
        return (return_var)
    else:
        return (fail_var)

## getting all responses
    
def get_all_responses(var_name):
	list0_, list1_ = [], intents['intents']
	for i in list1_:
		if i['tag'] == var_name:
			for s0_ in i['responses0']:
				for s1_ in i['responses1']:
					for s2_ in i['responses2']:
						list0_.append(s0_ + s1_ + s2_)
	return(list0_)
        

print('Bot is on, say hi!')

## remove this when you are done bridging gate between app and server
name = "Marcos"

res = 'temp'

# while True:
# 	add_to_database = get_all_responses('add to database now')
# 	message = spell(input('').lower())

# 	## test variable, remove before release
# 	if message == 'break':
# 		print('turning off')
# 		break

#     ## bot training and storing after training
# 	elif res == var_finder(add_to_database, res, res, False):
# 		message_0 = input('').lower()
# 		list_temp = []
# 		list_temp.append(message)
# 		list_temp.append(message_0)
# 		res = get_response([{'intent': 'training done', 'probability': '1'}], intents)[0]
# 		followups = get_response([{'intent': 'training done', 'probability': '1'}], intents)[1]
# 		print(res)
# 		if followups != "":
# 			print(followups)

# 	else:
# 		ints = predict_class(message)
#         ## bot is certain enough its the correct response 
# 		print(float(ints[0]['probability']))
# 		if float(ints[0]['probability']) > .9:
# 			res = get_response(ints, intents)[0]
# 			followups = get_response(ints, intents)[1]
#         ## bot is uncertain answer is correct result
# 		else:
# 			print(message)
# 			res = get_response([{'intent': 'bot uncertain', 'probability': '1'}], intents)[0]
# 			followups = get_response([{'intent': 'bot uncertain', 'probability': '1'}], intents)[1]
# 		print(res)
# 		if followups != "":
# 			print(followups)

Bot is on, say hi!


In [23]:
def chat(message):
	add_to_database_ = get_all_responses('add to database now')
	name = "Marcos"
	res = 'temp'
    ## bot training and storing after training
	if res == var_finder(add_to_database_, res, res, False):
		message_0 = input('').lower()
		list_temp = []
		list_temp.append(message)
		list_temp.append(message_0)
		res = get_response([{'intent': 'training done', 'probability': '1'}], intents)[0]
		followups = get_response([{'intent': 'training done', 'probability': '1'}], intents)[1]
		print(res)
		if followups != [""]:
			print(followups)

	else:
		ints = predict_class(message)
        ## bot is certain enough its the correct response 
		if float(ints[0]['probability']) > .9:
			res = get_response(ints, intents)[0]
			followups = get_response(ints, intents)[1]
        ## bot is uncertain answer is correct result
		else:
			res = get_response([{'intent': 'bot uncertain', 'probability': '1'}], intents)[0]
			followups = get_response([{'intent': 'bot uncertain', 'probability': '1'}], intents)[1]
		print(res)
		if followups != [""]:
			print(followups)
message = spell(input('').lower())
chat(message)

 hi


Hi Marcos! So what do you need?
['I need help with a question.', 'I want to give feedback.', "I'm new!", 'I need help with something else.']
